In [ ]:
from multiprocessing import Pool
from tqdm.auto import tqdm
import cProfile, pstats
import numpy as np
from numba import njit, prange
%load_ext snakeviz

In [ ]:
#test pool with gen
items = ['a','b','c']
vals = (['a',[100,101,102]],['b',[200,201,202,203]],['c',[300,301]]) # list-based
j = iter(range(len(vals)))
k={}


In [ ]:
def test(item):
    kk = next(j)
    k[kk] = item
    for val in enumerate(item[1]):
        print(f'\n{item[0]}[{i}] has {val}')

In [ ]:
@njit(parallel=True)
def test_nb(item):
    for val in prange(item.shape[0]):
        print(f'\n {val}')

In [ ]:
def dtest(items):
    print(f'\n[{items[0]}] has {items[1]}')

In [ ]:
vs= {k:v for k,v in vals}
v_ks = {i:k for i,k in enumerate(vs.keys())}
val_count = sum( len(v) for v in list(vs.values()))
val_count

In [ ]:
ixn_keys = [[(ik,nk) for nk,_ in enumerate(vs[ v_ks[ik] ])] for ik in tuple(v_ks.keys())]
dummy_dict = {}
for inkl, nl in zip( ixn_keys,vs.values() ):
    print(f'associate members of {nl} to {inkl}')
    for ink,n in zip(inkl,nl):
        print(f'{ink} <-- {n}')
        dummy_dict[ink] = n #dict-based

In [ ]:
(a,b), c = list(dummy_dict.items())[0]
a,b,c

In [ ]:
def test_():
    pool = Pool()
    with pool:
        tuple( tqdm( pool.imap( test_nb, vals ), total=3) )

In [ ]:
def dtest_():
    pool = Pool()
    with pool:
        tuple( tqdm( pool.imap( dtest, dummy_dict.items() ), total=3) )

In [ ]:
def profile_f(f):
    profiler = cProfile.Profile()
    profiler.enable()
    f()
    profiler.disable()
    stats = pstats.Stats(profiler).sort_stats('tottime')
    return stats

In [ ]:
# lt = profile_f(test_).print_stats()
# dt = profile_f(dtest_).print_stats()


In [ ]:
%snakeviz dtest_()

### Potential Chip Dimensions

In [ ]:
from math import floor, ceil
to_cm = lambda x: round(x*2.7/10,1)

img_w = 9504
img_h = 6336
nws = []
nhs = []
# thr = 0.001
for s in range(floor(img_w/40),img_w):
    w = s
    h = w * (2/3)
    if h/3*2 % 1 == 0:
        min_ov,max_ov = floor(w*.10), ceil(w*.66)
        for o in range(min_ov,max_ov):
            # how many blocks of side x side fit on width and height of the image
            nw = (img_w - o)/(s - o)
            nh = (img_h - o)/(s - o)
            if nw % 1 == 0 and nh % 1 == 0:
                nws.append(nw)
                nhs.append(nh)
                print(f'{int(nw-1)}x{int(nh-1)} colxrow_tiles/img, {s} px/side or ~{to_cm(s)}cm/tile_side, overlapping by {o}px or {round(o/s*100,1)}% or ~{to_cm(o)}cm ')
                print(f'{nw*nh} tiles per image')
                print('----------------------------------------------------------------------')
print(f'nws: {len(nws)}, nhs: {len(nhs)}')

### Rasterio Tiler (Not concurrent processing)

In [ ]:
import os
import rasterio as rio
from rasterio import windows
from itertools import product

jp = lambda *p: os.path.join(*p)

sites = ['CND']

in_paths = {site:f'E:\\Georeferenced\\{site}' for site in sites}
in_flnms = os.listdir(in_paths['CND'])

out_paths = {site:f'E:\\Tiled\\{site}' for site in sites}
out_flnm = 'tile_{}-{}.tif' # tile_IMG_jxi


#64tiles/block 508px (137.2cm) wide -- ovlp 128px (~25.2%, 34.6cm)

# https://gis.stackexchange.com/questions/285499

def get_overlapping_tiles(ds, width=508, height=508, ovlp=128):
    ncols, nrows = [ds.meta[v] for v in ('width','height')]
    xoff, yoff = [v-ovlp for v in (width, height)]
    offsets = product(*[range(0,direction,offset) for direction,offset in zip( (ncols,nrows), (xoff,yoff) )])
    big_window = windows.Window(col_off=0, row_off = 0, width=ncols, height=nrows)
    for col_off, row_off in offsets:
        window = windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window,ds.transform)
        yield window, transform

with rio.open( jp(in_paths['CND'],in_flnms[0]) ) as inds:
    tile_width = tile_height = 508
    meta = inds.meta.copy()
    img_nm = os.path.basename(in_flnms[0])
    img_id = img_nm.replace('_DSC','').replace.split('.')[0]
    

    for window, transform in get_overlapping_tiles(inds):
        print(window)
        meta['transform'] = transform
        meta['width'], meta['height'] = window.width, window.height
        os.makedirs(out_paths['CND'],exist_ok=True)
        outpath = jp(out_paths['CND'],out_flnm.format(*[int(loc) for loc in (window.col_off, window.row_off)]))
        with rio.open(outpath, 'w', **meta) as outds:
            outds.write(inds.read(window=window))